In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#common libs
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import DataLoader
from torchinfo import summary
import math
from easydict import EasyDict as edict


In [3]:
#mtr modules
from mtr.datasets import build_dataloader
from mtr.config import cfg, cfg_from_yaml_file
from mtr.utils import common_utils

In [4]:
cfg_from_yaml_file("/code/jjiang23/csc587/KimchiVision/cfg/kimchiConfig.yaml", cfg)
logger = common_utils.create_logger("/files/waymo/damon_log.txt", rank=0)
args = edict({
    "batch_size": 1,
    "workers": 32,
    "merge_all_iters_to_one_epoch": False,
    "epochs": 5,
    "add_worker_init_fn": False,
    
})

In [5]:
#prepare data
train_set, train_loader, train_sampler = build_dataloader(
    dataset_cfg=cfg.DATA_CONFIG,
    batch_size=args.batch_size,
    dist=False, workers=args.workers,
    logger=logger,
    training=True,
    merge_all_iters_to_one_epoch=args.merge_all_iters_to_one_epoch,
    total_epochs=args.epochs,
    add_worker_init_fn=args.add_worker_init_fn,
)

test_set, test_loader, sampler = build_dataloader(
        dataset_cfg=cfg.DATA_CONFIG,
        batch_size=args.batch_size,
        dist=False, workers=args.workers, logger=logger, training=False
)

2025-06-08 08:55:33,032   INFO  Start to load infos from /files/waymo/code/MTR/data/waymo/processed_scenarios_training_infos.pkl


2025-06-08 08:55:39,146   INFO  Total scenes before filters: 243401
2025-06-08 08:55:45,576   INFO  Total scenes after filter_info_by_object_type: 243401
2025-06-08 08:55:45,585   INFO  Total scenes after filters: 243401
2025-06-08 08:55:45,587   INFO  Start to load infos from /files/waymo/code/MTR/data/waymo/processed_scenarios_val_infos.pkl
2025-06-08 08:55:47,924   INFO  Total scenes before filters: 22089
2025-06-08 08:55:48,514   INFO  Total scenes after filter_info_by_object_type: 22089
2025-06-08 08:55:48,544   INFO  Total scenes after filters: 22089


In [ ]:
from lstm.lstm import MotionLSTM
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MotionLSTM().to(device)

home of the LSTM /home/dlin42/KimchiVision, change when needed


In [7]:
batch = next(iter(train_loader))

home of the LSTM /home/dlin42/KimchiVision, change when needed


Traceback (most recent call last):
  File "/files/waymo/miniconda3/envs/lstm/lib/python3.8/multiprocessing/queues.py", line 245, in _feed
    send_bytes(obj)
  File "/files/waymo/miniconda3/envs/lstm/lib/python3.8/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/files/waymo/miniconda3/envs/lstm/lib/python3.8/multiprocessing/connection.py", line 411, in _send_bytes
    self._send(header + buf)
  File "/files/waymo/miniconda3/envs/lstm/lib/python3.8/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe


In [8]:
batch["input_dict"].keys()

dict_keys(['scenario_id', 'obj_trajs', 'obj_trajs_mask', 'track_index_to_predict', 'obj_trajs_pos', 'obj_trajs_last_pos', 'obj_types', 'obj_ids', 'center_objects_world', 'center_objects_id', 'center_objects_type', 'obj_trajs_future_state', 'obj_trajs_future_mask', 'center_gt_trajs', 'center_gt_trajs_mask', 'center_gt_final_valid_idx', 'center_gt_trajs_src', 'map_polylines', 'map_polylines_mask', 'map_polylines_center', 'static_map_polylines', 'static_map_polylines_mask'])

In [9]:
input = batch["input_dict"]
obj_trajs = input["obj_trajs"]
obj_pos = input["obj_trajs_pos"]
obj_last_pos = input["obj_trajs_last_pos"]
obj_type = input["obj_types"] # car, bicycycle, pedestrian
obj_trajs_mask = input['obj_trajs_mask']
obj_of_interest = input['track_index_to_predict']

In [10]:
num_center_objects, num_objects, num_timestamps, num_attrs = obj_trajs.shape

In [13]:
static_map_polylines=input["static_map_polylines"].to('cuda')  # (batch_size, num_polylines, num_points_each_polyline, 7)
static_map_polylines_mask=input["static_map_polylines_mask"].to('cuda') # (batch_size, num_polylines, num_points_each_polyline)

In [42]:
obj_trajs_mask[:, 1, :].sum(dim=1)

tensor([11, 11, 11, 11, 11, 11, 11, 11])

In [80]:
from einops import rearrange, repeat

torch.nonzero(obj_trajs_mask[0, 0, :])[-1]

tensor([10])

In [19]:
model._print_batch(batch)

home of the LSTM /home/dlin42/KimchiVision, change when needed
Key: scenario_id, Val: (8,)
Key: obj_trajs, Val: torch.Size([8, 24, 11, 29])
Key: obj_trajs_mask, Val: torch.Size([8, 24, 11])
Key: track_index_to_predict, Val: torch.Size([8])
Key: obj_trajs_pos, Val: torch.Size([8, 24, 11, 3])
Key: obj_trajs_last_pos, Val: torch.Size([8, 24, 3])
Key: obj_types, Val: (24,)
Key: obj_ids, Val: (24,)
Key: center_objects_world, Val: torch.Size([8, 10])
Key: center_objects_id, Val: (8,)
Key: center_objects_type, Val: (8,)
Key: obj_trajs_future_state, Val: torch.Size([8, 24, 80, 4])
Key: obj_trajs_future_mask, Val: torch.Size([8, 24, 80])
Key: center_gt_trajs, Val: torch.Size([8, 80, 4])
Key: center_gt_trajs_mask, Val: torch.Size([8, 80])
Key: center_gt_final_valid_idx, Val: torch.Size([8])
Key: center_gt_trajs_src, Val: torch.Size([8, 91, 10])
Key: map_polylines, Val: torch.Size([8, 768, 20, 9])
Key: map_polylines_mask, Val: torch.Size([8, 768, 20])
Key: map_polylines_center, Val: torch.Size([8

In [97]:
map_object_features = model(batch)


home of the LSTM /home/dlin42/KimchiVision, change when needed


In [98]:
map_object_features

(tensor([[0.1460, 0.1754, 0.1473, 0.1977, 0.1649, 0.1687],
         [0.1497, 0.1783, 0.1467, 0.1919, 0.1659, 0.1674],
         [0.1492, 0.1807, 0.1424, 0.1930, 0.1639, 0.1708],
         [0.1486, 0.1841, 0.1581, 0.1835, 0.1689, 0.1569],
         [0.1565, 0.1822, 0.1454, 0.1915, 0.1676, 0.1569],
         [0.1504, 0.1811, 0.1408, 0.1962, 0.1664, 0.1651],
         [0.1511, 0.1813, 0.1517, 0.1830, 0.1671, 0.1658],
         [0.1549, 0.1819, 0.1452, 0.1816, 0.1722, 0.1641]], device='cuda:0',
        grad_fn=<SoftmaxBackward0>),
 tensor([[[[-3.7919e-03, -8.2558e-02,  4.3923e-02, -1.1123e-02],
           [-2.5621e-02, -9.0568e-05,  5.1938e-02,  3.4283e-02],
           [ 1.3992e-02, -2.1788e-02, -5.3575e-02, -2.5558e-02],
           ...,
           [-2.6081e-02, -3.9168e-02,  1.8734e-02,  2.5749e-02],
           [-3.1466e-03, -8.7790e-02,  4.3929e-02,  2.5645e-03],
           [ 7.3851e-03,  1.7852e-02, -8.1599e-03, -2.6107e-02]],
 
          [[ 5.2880e-02, -6.3428e-02,  6.3580e-03,  7.3376e-02],

In [93]:
obj_trajs_mask

tensor([[[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         ...,
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True]],

        [[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         ...,
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True]],

        [[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         ...,
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True]],

In [89]:
obj_trajs_mask.sum(dim=2).shape

torch.Size([8, 24])